In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/causal/public/ground_truth.csv", names=["label"])

In [3]:
df["filename"] = df.index.map(lambda x: f"{x}.png")

In [4]:
df["y"] = df.label.apply(str)

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-04-27 00:45:04.828015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 00:45:04.828034: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
datagen = ImageDataGenerator(rescale=1./255.)#,validation_split=0.2)

In [7]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory="../data/causal/public/",
    x_col="filename",
    y_col="y",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="binary",
    color_mode="grayscale",
    target_size=(320,240)
)
#valid_generator=datagen.flow_from_dataframe(
#    dataframe=df,
#    directory="../data/causal/public/",
#    x_col="filename",
#    y_col="y",
#    subset="validation",
#    batch_size=32,
#    seed=42,
#    shuffle=True,
#    class_mode="binary",
#    color_mode="grayscale",
#    target_size=(320,240)
#)

Found 1000 validated image filenames belonging to 2 classes.


In [8]:
model = keras.Sequential(
    [
        keras.Input(shape=(320, 240, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 318, 238, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 159, 119, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 157, 117, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 78, 58, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 76, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 38, 28, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 68096)             0

2022-04-27 00:45:05.674441: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-27 00:45:05.674459: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-27 00:45:05.674481: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eka-thinkpad): /proc/driver/nvidia/version does not exist
2022-04-27 00:45:05.674677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [10]:
model.fit(train_generator, batch_size=32, epochs=12) #, validation_data=valid_generator

2022-04-27 00:45:05.876765: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/12
32/32 [==============================] - 35s 1s/step - loss: 1.7078 - accuracy: 0.5100
Epoch 2/12
32/32 [==============================] - 33s 1s/step - loss: 0.7133 - accuracy: 0.5460
Epoch 3/12
32/32 [==============================] - 37s 1s/step - loss: 0.8559 - accuracy: 0.5710
Epoch 4/12
32/32 [==============================] - 34s 1s/step - loss: 0.6322 - accuracy: 0.6500
Epoch 5/12
32/32 [==============================] - 35s 1s/step - loss: 0.6069 - accuracy: 0.6570
Epoch 6/12
32/32 [==============================] - 36s 1s/step - loss: 0.5931 - accuracy: 0.7090
Epoch 7/12
32/32 [==============================] - 35s 1s/step - loss: 0.5352 - accuracy: 0.7350
Epoch 8/12
32/32 [==============================] - 33s 1s/step - loss: 0.5255 - accuracy: 0.7550
Epoch 9/12
32/32 [==============================] - 35s 1s/step - loss: 0.4901 - accuracy: 0.7720
Epoch 10/12
32/32 [==============================] - 35s 1s/step - loss: 0.3906 - accuracy: 0.8220
Epoch 11/12
32/32 [

In [11]:
test = pd.DataFrame(data=np.arange(1000), columns=["index"])

In [12]:
test["filename"] = df.index.map(lambda x: f"{x}.png")

In [13]:
testgen = ImageDataGenerator(rescale=1./255.)

test_generator=testgen.flow_from_dataframe(
    dataframe=df,
    directory="../data/causal/private/",
    x_col="filename",
    y_col="y",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="binary",
    color_mode="grayscale",
    target_size=(320,240)
)

Found 1000 validated image filenames belonging to 2 classes.


In [14]:
pred = model.predict(test_generator)

In [15]:
(pred > 0.5).astype(np.int).reshape(-1).shape

(1000,)

In [16]:
test["direction"] = (pred > 0.5).astype(np.int)

In [17]:
test

,index,filename,direction
0,0,0.png,0
1,1,1.png,0
2,2,2.png,0
3,3,3.png,1
4,4,4.png,1
...,...,...,...
995,995,995.png,0
996,996,996.png,1
997,997,997.png,0
998,998,998.png,0


In [18]:
test[["index", "direction"]].to_csv("../predictions/t4_img_clf.csv", index=False)